In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import statistics
from sklearn import svm
#from scipy.spatial.transform import Rotation
from scipy.signal import find_peaks, butter, filtfilt
from scipy.io import wavfile
from IPython.display import Audio
import time
import threading
import ble_itf

In [7]:
#RETRAIN!!!!!!!!!!!!!!!!!!!!!!
def movement_classify():
    X = [[2.2, 0.5428574122182267], [2.2, 0.5404589046459198], [2.4, 0.5525351030388725], [2.4, 0.541671541360155], [3.0, 1.3630733939992992], [2.8, 0.9675008316355763], [2.8, 0.9177557494006532], [3.0, 0.8227610468680422], [2.2, 0.48542657722277854], [2.4, 0.4515375846000618], [2.2, 0.6913393373331244], [2.2, 0.5147064781432921], [2.4, 0.5659609493986341], [2.6, 0.5670707115545206], [2.4, 0.5530952013135776], [2.4, 0.5433910570786663], [2.0, 0.3427726924350805], [2.2, 0.35785369267888106], [2.2, 0.32088708053300724], [2.0, 0.35820110407927225], [1.8, 0.2441267727566469], [2.2, 0.2560974573163057], [2.4, 0.29013040991723454], [2.6, 0.26375776763130115], [2.0, 0.3682096009760422], [2.2, 0.3106402410275752], [2.0, 0.34740369528865933], [2.0, 0.3684995397804337], [0.0, 0.09550067361576874], [0.0, 0.05741004333376792], [0.0, 0.04622048628301063], [0.0, 0.15054380757983327], [4.2, 1.0641837451676484], [4.0, 3.7371110017109523], [4.0, 0.8438362903023318], [3.8, 1.0766827098271423], [3.4, 1.3490433320503434], [3.4, 1.225342605407773], [3.4, 1.4298454144233401], [3.4, 1.333955893999703], [3.4, 1.2653283359140188], [3.2, 1.1744916325046808], [3.2, 1.2062698061964339], [3.2, 1.2327780823632866], [3.2, 1.8176819110610922], [3.2, 2.462146601677347], [3.2, 2.7366759282369366], [3.0, 2.50136941277341]]
    Y = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X, Y)
    five_sec_df = pd.DataFrame(data={})

    while True:
        imu_buf = ble_itf.imu_buf  
        temp_dict = {                           #.......................................................
            'a_xyz': imu_buf['a_xyz'][-476:]
        }
        if len(imu_buf['a_xyz']) > 476:
            five_sec_df = five_sec_df[:min(len(five_sec_df),476*4)]
            five_sec_df += pd.DataFrame.from_dict(temp_dict)
            if len(five_sec_df) >= 476*5:     
                X_cur = imu_steps_freq_and_peak_height(dict(five_sec_df),5)
                print(lin_clf.predict(X_cur))
                time.sleep(1)
    

In [3]:
df = pd.read_csv('imu_jogging_ed_70.csv')
print(type(df))
df_len = len(df)
imu_5s_bufs = []
imu_5s_bufs.append(dict(df[0:int(df_len/4)]))
imu_5s_bufs.append(dict(df[int(df_len/4):int(df_len/2)]))
imu_5s_bufs.append(dict(df[int(df_len/2):int(df_len*3/4)]))
imu_5s_bufs.append(dict(df[int(df_len*3/4):]))

NameError: name 'pd' is not defined

In [4]:
def imu_steps_freq_and_peak_height(imu_buf, time_interval):

    steps = 0
    avg_peak_height = 0

    if len(imu_buf['a_xyz']) >= 476 * time_interval - 60:    #imu sample rate = 476 Hz; 60 4-byte points taken at a time for ble; we can get the minimum points recorded by time_interval seconds
        a_xyz = imu_buf['a_xyz']

        # low pass filter
        b, a = butter(3, 0.048, 'low')
        y = filtfilt(b, a, a_xyz)

        # peak detection
        y -= np.min(y)
        peaks = find_peaks(y, height = 0)
        
        # find mean (use median in the future in case of outliers) peak height and number of steps
        steps = 0
        for peak_height in peaks[1]['peak_heights']:
            avg_peak_height += peak_height
            if peak_height >= 0.2:             # required minimum height for a peak
                steps += 1
        avg_peak_height /= len(peaks[0])
        
        #print(steps,"avg_peak_height: ", avg_peak_height)
        #print('min peak height: ', min(peaks[1]['peak_heights']))
        #print('max peak height: ', max(peaks[1]['peak_heights']))
        
    return [steps/time_interval, avg_peak_height]


In [5]:
ble_itf.mic_buf = []
ble_itf.imu_buf = {
	# 'ax' : [],
	# 'ay' : [],
	# 'az' : [],
	'a_xyz' : []
	'wx' : [],
	'wy' : [],
	'wz' : [],
	'mx' : [],
	'my' : [],
	'mz' : [],
}
# walking_thread = threading.Thread(target = movement_classify, name = 'walking_thread')
# walking_thread.start()
await ble_itf.call()
# global stop_thread
# stop_thread = 1
# walking_thread.join()

NameError: name 'ble_itf' is not defined